<a href="https://colab.research.google.com/github/deregenboogm/NLP_coronavirus_project/blob/master/data_frames/risk_assessments_with_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def get_risk_reports():
  ## Define url
  url = "https://www.ecdc.europa.eu/en/taxonomy/term/1295/feed"

  ## Request to access the url
  r = requests.get(url)
  
  ## Parsing the content
  soup = BeautifulSoup(r.content, features="xml") 

  ## Find items in the content
  items = soup.find_all("item")

  ## Optain a list of report items with their features  
  news_items=[]
     
  for item in items:
    if "COVID" in item.title.text:    ### searching in title text gives better results for related texts at this web-page
      news_item = {}                     
      news_item["title"] = item.title.text
      news_item["date"] = item.pubDate.text
      news_item["date"] = pd.to_datetime(news_item["date"])
      news_item["date"] = news_item["date"].isoformat().split('T')[0]
      news_item["link"] =  item.link.text
      news_item["description"] = item.description.text
      news_item["summary"] = []
      
      ### Optain texts through accessing web links 
      for link in item.link:
        res = requests.get(link)
        soup = BeautifulSoup(res.content, "html.parser")

        article = soup.find_all("div", class_ = "text-image")

        executive_summary = []
              
        for pr in article:
          paragraphs = pr.find_all("p")
          for i in paragraphs:
            joined_p = "".join(i.text)
            joined_p = joined_p.replace(".", ". ")
            joined_p = joined_p.replace(" '", "'")
            executive_summary.append(joined_p)
                        
        executive_summary = ''.join(map(str, executive_summary))      

        ##### Update each item dictionary through using the text summarization function that has been already created above
        summ = generate_summary(executive_summary)
        news_item["summary"] = summ         
      
      ### Append each item dictionary to the general item list(results)
      news_items.append(news_item)
       
  df = pd.DataFrame(news_items, index=None)

  # Saving the dataframe table
  df.to_csv("/content/drive/My Drive/Risk_assessment_reports.csv", index = False)

  print("Total number of COVID-19 risk assessment repots: " , len(news_items))
  print("The 'Risk_assessment_reports.csv' file has been successfully updated for today!")